# PV181 Seminar 02- RNG (python)

This notebook contains python code for several tasks treated in this seminar. 

# PRNG 

# Task 1: determinsm of PRNG

We will work with PRNG implemented in [random](https://docs.python.org/3/library/random.html) package. See first 4 methods (`seed, setstate, getstate, randbytes`) in the documentation. 
 1. Import **random** package.
 2. Generate (and print) 10 random bytes.  
 3. Print out bytes in hexadecimal form (use `.hex()` method of bytes). Execute cell 2x. </br> PRNG produced different results since  generation is not deterministic as it is seeded by time. 
 4. Use fixed `seed` and verify that generation is deterministic (generated bytes are always the same).     
 5. Verify that seed determines internal state of the generator. What is the internal state for `seed = 1`? 
 6. Use `seed = 1` and generate 10 bytes. Use the internal state from previous step 5. and generate the same 10 bytes - you should see same bytes.  
 7. **Attack**: The generator produced 16 bytes that will be used as AES key. The first half of is `73a9bef499bbf4dca4f2`. Find the rest of the key. </br> **Hint**: user used small seed.

In [2]:
import random 
rnd_bytes = random.randbytes(10)
print(rnd_bytes)
print(rnd_bytes.hex())

random.seed(2)
state = random.getstate()
print(random.randbytes(10).hex())

# # print(state)
# random.setstate(state)
# print(random.randbytes(10).hex())

b'\xc8\xaf[\xd9\x9f&z\x0er\x17'
c8af5bd99f267a0e7217
73a9bef499bbf4dca4f2


# LCG
Standard PRNG functions are very fast but also very weak. 
 * In python, PRNG [implemented](https://svn.python.org/projects/python/branches/release32-maint/Lib/random.py) in random module is [Mersenne Twister](https://en.wikipedia.org/wiki/Mersenne_Twister) with state formed by 625 32-bit integers. See size of the state in Task 1 above.
 * In other languages (C, Java, Rust) LCG is typically used. Internal state of LCG is **single** value (state) updated iterativelly as $$state = (state*a+c) \pmod m.$$ Overview of constants `a,c,m` used by the LCG for several languages can be found [here](https://en.wikipedia.org/wiki/Linear_congruential_generator).  
 <span style="color:red">In LCG, state (new or old) is typically returned as generated random value!!</span>

# Task 2: common rand PRNG  
Following code was taken from [ANSI C standard](http://www.open-std.org/jtc1/sc22/wg14/www/docs/n1256.pdf#page=324) and simplified to other portable implementation (according to implementation of [rand()](https://code.woboq.org/userspace/glibc/stdlib/random_r.c.html#__random_r)) of seeding function `srand` and function for generation `rand()`. 

```
static unsigned long int next = 1;

void srand(unsigned int seed)
{
    next = seed;
}

int rand(void) 
{
    return next = (next * 1103515245 + 12345) & 0x7fffffff;
}
```
 1. Implement pythonic version of ANSI C PRNG - implement it as `class PRNG` below. </br> Use constants $a=1103515245, c=12345, m=2^{31}$ from [LCG wikipedia](https://en.wikipedia.org/wiki/Linear_congruential_generator).  
 2. Generate 10 values $12345, 1406932606, ...$ with LCG seeded by 0.  
 3. Generate 10 values but use different seed so the sequence will start with $1406932606, ...$.  

In [19]:
class PRNG:
    def __init__(self):
        self.srand(1)

    def srand(self, seed):
        self.state = seed

    def rand(self):
        self.state = (self.state * 1103515245 + 12345) % 2**31
        return self.state

ansi_rand = PRNG()
ansi_rand.srand(0)
rnd_values = [ansi_rand.rand() for i in range (4)]
print(rnd_values)

[12345, 1406932606, 654583775, 1449466924]


 4. Every PRNG generates values in cycle i.e. generated sequence is periodic. Find the seed of the generator for which the generated sequence would be `[??, ??, 12345, 1406932606]`. Find previous two values (replaced by ??). </br>
 **Hint**:
 To revert this PRNG you can use constants $a^{-1}$ and $(c*a^{-1})$ instead of constants $a$ and $c$.  </br>
 Compute $a^{-1}$ and $(c*a^{-1})$ for ANSI C and generate (`[1406932606, 12345, ??, ??, seed]`).</br>
 In order to find backward LCG  it suffices to invert the update function: $$new\_state = old\_state*a+c \pmod m.$$   
 The inverse function can be computed as 
    $$
    \begin{align}
     old\_state &= (new\_state - c)/a \pmod m \\
                &= new\_state*(a^{-1}) - (c*a^{-1}) \pmod m \\
    \end{align}
    $$
  where $a^{-1} \pmod m$ can be computed using `*pow(a,-1,m)`. 
  5. Use LCG.py script and generate sequence of 10 numbers for java_util_random with `seed=1`. Folowing cmd will generate the sequence for you. Verify (google) that sequence is correct. </br>
  `python3 LCG.py -m 2**48 -a 25214903917 -c 11 -s 0 -l 16 -u 47 -n 10`
  
 

# Task 3: small state attack
 1. Use rng and generate random bytes which will be used below as AES key. 

In [30]:
from cryptography.hazmat.primitives import hashes

def SHA1(message: bytes):
    digest = hashes.Hash(hashes.SHA1())
    digest.update(message)
    return digest.finalize() 

def SHA256(message: bytes):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(message)
    return digest.finalize() 

class PRNG:
    def __init__(self):
        self.srand(b'0x00')

    def srand(self, seed):
        self.state = seed

    def rand_bytes(self, num_bytes=10):
        rnd = SHA1(self.state)[:num_bytes]
        self.state = SHA256(self.state)[:1]
        return bytes(rnd)

rng = PRNG()
rng.srand(b'0x12')
rng.rand_bytes(4)

b'\xd2\xb9\xb2A'

 2. Use `encrypt_ECB` and `encrypt_ECB` to encrypt and decrypt arbitrary short(16B) message.

In [31]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

def encrypt_ECB(key, msg):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    enc = cipher.encryptor()
    ct = enc.update(msg) + enc.finalize()
    return ct 

def encrypt_ECB(key, ct):
    cipher = Cipher(algorithms.AES(key),  modes.ECB())
    dec = cipher.decryptor()
    pt = dec.update(ct) + dec.finalize()
    return pt

K = os.urandom(16)
ct = encrypt_ECB(K, b'arbitrarymessage')
pt = decrypt_ECB(K, ct)
print(ct)

b'\xaf\x8c\xd1W\xb7\xda\xf2\xe4\x8c*t}\x7f\xb7\xdd\x1d'


 3. **Attack**: Find all possible 16 byte blocks `rng` can produce and decrypt the ciphertext `ct=af8cd157b7daf2e48c2a747d7fb7dd1d`.
    

In [36]:
ct = b'\xaf\x8c\xd1W\xb7\xda\xf2\xe4\x8c*t}\x7f\xb7\xdd\x1d'

# TRNG

# Sources: dev/random, dev/urandom
These two files provide secure way to generate random bytes! 
Reading from dev/urandom can by done using following functions: 

In [9]:
import os
import secrets 
os.urandom(10)
print(secrets.token_bytes(10).hex())
print(os.getrandom(10).hex())

b'^\xc3\x16\xf3\xe1\xf5\r\xee\xff,'
b"Lrmg\xa8x'\x05\xc7j"


Files **dev/random**, **dev/urandom** can be also opened as binary file for reading.  
Then you can read specified number of bytes e.g. 10. 

In [9]:
random_source = open("/dev/random", "rb")
random_source.read(10)

b'b\xd0\xbd\xb4\xe2\xd1\xae`cL'

# Task 5: Testing correlation of bits
 1. Implement function `histogram(rnd_bytes, i, j)` that computes histogram of combination of bits (`i`-th and `j`-th bits of each byte). The function should return 4 frequencies for all bytes in `rnd_bytes` array. Frequencies will correspond to counts of how many bytes have `i`-th and `j`-th bit equal to combination 00,01,10 or 11.


In [37]:
import os
def histogram(rnd_bytes, i, j):
    hist = {0:0, (1 << i):0, (1 << j):0, (1 << i) + (1 << j): 0}
    mask = (1 << i) + (1 << j)
    for byte in rnd_bytes:
        hist[byte & mask] += 1
    return hist 



 2. Verify that for arbitrary params `i,j` and size of generated block the frequencies are roughly equal.  

In [38]:
rnd_bytes = os.urandom(1000)
histogram(rnd_bytes, 0, 1)

{0: 270, 1: 234, 2: 247, 3: 249}

 3. Generate random bytes using ansi_rand = generate integers, apply modulo, transform to bytes (use `bytes()`).
 4. Find params `i,j` where all the frequencies are exacly the same.  
 <span style="color:red"> Generator with such perfect results is also problematic! </span>   
 Can we predict some (next) bits with better probability than 50% ?

In [39]:
ansi_rand.srand(6)
rnd_bytes = bytes([ansi_rand.rand() % 256 for i in range (1000)])
histogram(rnd_bytes, 0, 1)

{0: 250, 1: 250, 2: 250, 3: 250}